### Construction de cartes interactives avec folium

In [4]:
# espace de travail
pwd

'C:\\Users\\HP\\Downloads'

In [8]:
# On exécute le code qui retraite les données

%run "./Pré_traitements_carto.ipynb"

In [ ]:
# Installation du package nécessaire
pip install folium

In [ ]:
# Calcul des points centraux des communes afin de centrer la carte
data_geo2 = communes_IDF[['INSEE_COM', 'geometry']]
data_geo2['Center_point'] = data_geo2['geometry'].centroid

In [ ]:
data_geo2.head()

In [ ]:
#Extraction de la latitude et de la longitude de ces points centraux
data_geo2["lat"] = data_geo2.Center_point.map(lambda p: p.x)
data_geo2["long"] = data_geo2.Center_point.map(lambda p: p.y)

In [ ]:
data_geo2.head()

In [ ]:
# Première visualisation

import folium

# Calcul du centre de la carte
data_geo2 = communes_IDF[['INSEE_COM', 'geometry']]
data_geo2['Center_point'] = data_geo2['geometry'].centroid
#Extract lat and lon from the centerpoint
data_geo2["lat"] = data_geo2.Center_point.map(lambda p: p.x)
data_geo2["long"] = data_geo2.Center_point.map(lambda p: p.y)

# Sortie de la carte primaire
m = folium.Map(location=[data_geo2['long'].mean(), data_geo2['lat'].mean()],
               zoom_start=9, tiles='cartodbpositron')
# m 
# (Visualisation non nécessaire)

In [ ]:
# Première carte relative aux revenus

folium.Choropleth(
    geo_data=communes_IDF,
    data = data_used,
    #data = f_data,
    columns = ['INSEE_COM', 'MED16'],
    key_on = 'feature.properties.INSEE_COM',
    nan_fill_color='White',
    fill_color="YlGn",
    fill_opacity=0.8,
    line_opacity=0.5,
    legend_name="Revenu médian (€)"
    ).add_to(m)
# Ajout des infos bulles sur la carte

folium.features.GeoJson(
                    data=data_used,
                    name='Revenu median des ménages',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['NOM',
                                'NBMENFISC16',
                                'NBPERSMENFISC16',
                                'MED16'
                               ],
                        aliases=["<br>Commmune :",
                                 "<br>Nombre de ménages :",
                                 "<br>Population :",
                                 "<br>Revenu médian :"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(m)   

m

A ce stade, nous souhaitons afficher deux types d'informations sur une même carte interactive
-  La première affichera les données socio-démographiques relatives aux communes
-  La seconde renseignera sur les DPE dans les communes

In [ ]:
data_used['Energ2']=1-data_used['Energ']
data_used.head()

On construit la carte avec toutes ses thématiques. Quatre étapes sont nécessaires

In [ ]:
'''
Etape 1: On créée la carte de base et les deux couches séparément, 
Mais sans information au départ
'''
carte_fin = folium.Map(location=[data_geo2['long'].mean(), data_geo2['lat'].mean()],
               zoom_start=9.2, tiles=None)
cart_demo = folium.FeatureGroup(name='Données démographiques',overlay=False).add_to(carte_fin)
cart_dpe = folium.FeatureGroup(name='Diagnostics de performance énergétique',overlay=False).add_to(carte_fin)

In [ ]:
'''
Etape 2
On ajoute la première couche de carte à la carte "cart_demo
'''

# La carte choropleth
echelle = (f_data['MED16'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
nouv = folium.Choropleth(
    geo_data=communes_IDF,
    #geo_data=communes_IDF,
    data = data_used,
    columns = ['INSEE_COM', 'MED16'],
    key_on = 'feature.properties.INSEE_COM',
    fill_color='YlOrRd',
    nan_fill_color = 'White', # Couleur pour les données manquantes
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Revenu médian (€)",
    line_color = 'black',
    highlight = True
    ).geojson.add_to(cart_demo)

# Ajout des infos-bulles
folium.features.GeoJson(
                    data=data_used,
                    name='Revenu median des ménages',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['NOM',
                                'NBMENFISC16',
                                'NBPERSMENFISC16',
                                'MED16'
                               ],
                        aliases=["<br>Commmune :",
                                 "<br>Nombre de ménages :",
                                 "<br>Population :",
                                 "<br>Revenu médian :"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(nouv)

In [ ]:
'''
Etape 3:
On ajoute la deuxième couche choroplèthe à la carte "cart_demo
'''
# La carte choropleth
nouv2 = folium.Choropleth(
    geo_data=communes_IDF,
    data = data_used,
    columns = ['INSEE_COM', 'Energ2'],
    key_on = 'feature.properties.INSEE_COM',
    fill_color='YlGn',
    nan_fill_color = 'White', # Couleur pour les données manquantes
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Classe d'énergie majoritaire",
    line_color = 'black',
    highlight = True
    ).geojson.add_to(cart_dpe)

# Ajout des infos-bulles
folium.features.GeoJson(
                    data=data_used,
                    name="Proportion de ménages énergivores",
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['NOM',
                                'Energ',
                                'classe_consommation_energie_A',
                                'classe_consommation_energie_B',
                                'classe_consommation_energie_C', 
                                'classe_consommation_energie_D',  
                                'CE_Maj'
                               ],
                        aliases=["<br>Commmune :",
                                 "<br>Proportion ménages énergivores :",
                                 "<br>Proportion de A :",
                                 "<br>Proportion de B :",
                                 "<br>Proportion de C :",
                                 "<br>Proportion de D :",
                                 "<br>Classe majoritaire :",
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(nouv2)

In [ ]:
'''
Etape 4:
On ajoute des composantes à la carte: des tuiles et une barre de controle
'''
folium.TileLayer('cartodbdark_matter',overlay=True,name="View in Dark Mode").add_to(carte_fin)
folium.TileLayer('cartodbpositron',overlay=True,name="Viw in Light Mode").add_to(carte_fin)
folium.LayerControl(collapsed=False).add_to(carte_fin)

In [ ]:
# Affichage de la carte
carte_fin